In [1]:
import requests
import time
from bs4 import BeautifulSoup
import pandas as pd
import warnings
import datetime
import requests
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import csv
import io
import json
warnings.filterwarnings("ignore")

In [2]:
headers={
    'user-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.125 Safari/537.36'
    }

request_url='http://newhouse.591.com.tw/home/housing/info?hid=119282'
res=requests.get(request_url, headers = headers)
bs=BeautifulSoup(res.text,'html.parser')


In [3]:
# input 物件網址 撈取網頁資料 return 欄位 
def getData(url):
    request_url='https://newhouse.591.com.tw/home/housing/info?hid='+str(url).strip()
    headers={
    'user-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.125 Safari/537.36'
    }
    res=requests.get(request_url, headers = headers)
    #bs=BeautifulSoup(res.text,'html.parser')

    if res.status_code == 200:
        bs=BeautifulSoup(res.text,'html.parser')
        #先宣告變數為NULL 若無撈到資料則寫入NULL
        title='NULL'
        addr='NULL'
        htype='NULL'
        htype2='NULL'
        htype3='NULL'
        htype4='NULL'
        htype5='NULL'
        htype6='NULL'
        htype7='NULL'
        htype8='NULL'
        htype9='NULL'
        htype10='NULL'
        htype11='NULL'

        # 利用 beautfiulsoup 的 find function 利用 css selector 定位 並撈出指定資料
        title=bs.find('h1').text
        addr=bs.find("dl", {'class':"clearfix"}).findNext("dd").text.strip()
        htype=bs.find("dt", text="建案類別：").findNext("dd").string.strip()
        htype2=bs.find("dt", text="建物形態：").findNext("dd").string.strip().replace(' ', '').replace('\n', '、')
        htype3=bs.find("dt", text="公開銷售：").findNext("dd").string.strip()
        htype4=bs.find("dt", text="基地地址：").findNext("dd").contents[0].string.strip()
        htype5=bs.find("dt", text="交屋屋況：").findNext("dd").string.strip()
        htype6=bs.find("div",{'class':"flex_5 stonefont"}).findNext('span',{'class':''}).text.strip().replace(' ', '')
        htype7=bs.find("dt", text="投資建設：").findNext("dd").string.strip()
        htype8=bs.find("dt", text="營造公司：").findNext("dd").string.strip()
        htype9=bs.find("dt", text="棟戶規劃：").findNext("dd").string.strip()
        htype10=bs.find("dt", text="樓層規劃：").findNext("dd").string.strip()
        htype11=bs.find("dt", text="用途規劃：").findNext("dd").string.strip()
                
        return title,addr,htype, htype2, htype3, htype4, htype5, htype6, htype7, htype8, htype9, htype10, htype11
    else:
        print('link expired:', url)
        return 404, 404, 404, 404, 404, 404, 404



def main(outputfile, rid, sid):
   
   with io.open(outputfile, "w", encoding="utf-8") as csvfile:
        writer = csv.writer(csvfile)
        
        totalpages = 5
        print('Total pages: ', totalpages)

        for i in range(1, totalpages+1):
            request_url = "https://newhouse.591.com.tw/home/housing/search?rid="+str(rid)+"&sid="+str(sid)+"&page="+str(i)
            response = requests.get(request_url, headers = headers)
            response = response.json()
            items = response["data"]["items"]

            room_url_list=[] #存放網址list
            for key in items:
                id = key["hid"] # 每個物件的 id
                room_url_list.append(id)
            time.sleep(3)

            # ------------- write into csv ------------- #
#            info_url_list=[]
#            for url in list(set(room_url_list)):
#                url = url.replace("detail?", "info?")
#                info_url_list.append(url)
            for url in room_url_list:
                title,addr,htype, htype2, htype3, htype4, htype5, htype6, htype7, htype8, htype9, htype10, htype11 = getData(url)
                writer.writerow([title,addr,htype, htype2, htype3, htype4, htype5, htype6, htype7, htype8, htype9, htype10, htype11])
            # ------------------------------------------ #
            print(i/totalpages*100, '%') # print out 完成 %數

In [4]:
if __name__ == '__main__':
    # -------- configurable parameter -------- #
    output_file_name = 'tpe_rent_output.csv'
    # ---------------------------------------- #
    main(output_file_name, 1, 1)
    read_file = pd.read_csv (output_file_name)
    read_file.to_excel ('tpe_rent_output.xlsx', index = None, header=False)
    print('\nfinish!')

Total pages:  5
20.0 %
40.0 %
60.0 %
80.0 %
100.0 %

finish!
